In [ ]:
! pip install datasets sentence_transformers

     |████████████████████████████████| 311 kB 5.6 MB/s 
     |████████████████████████████████| 78 kB 6.4 MB/s 
     |████████████████████████████████| 67 kB 4.6 MB/s 
     |████████████████████████████████| 243 kB 51.0 MB/s 
     |████████████████████████████████| 133 kB 48.9 MB/s 
     |████████████████████████████████| 1.1 MB 36.5 MB/s 
     |████████████████████████████████| 3.5 MB 38.0 MB/s 
     |████████████████████████████████| 6.8 MB 40.0 MB/s 
     |████████████████████████████████| 1.2 MB 42.0 MB/s 
     |████████████████████████████████| 596 kB 44.6 MB/s 
     |████████████████████████████████| 895 kB 38.6 MB/s 
     |████████████████████████████████| 271 kB 49.8 MB/s 
     |████████████████████████████████| 144 kB 48.1 MB/s 
     |████████████████████████████████| 94 kB 1.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=faf959e06f5797685d735177353fc1464de3737960e0b33b68794089b4ceabdd
  Stored in di

In [ ]:
from datasets import load_dataset
data = load_dataset("squad_v2", split="train")
data

Downloading:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/9.55M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/801k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 130319
})

In [ ]:
from datasets import load_dataset
data = load_dataset('squad_v2', split="train")
data

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 130319
})

In [ ]:
from sentence_transformers import InputExample
from tqdm.auto import tqdm
train_data = []
for row in tqdm(data):
    train_data.append(InputExample(texts = [row["question"], row["context"]]))
train_data

  0%|          | 0/130319 [00:00<?, ?it/s]

 ...]

In [ ]:
from sentence_transformers import datasets
batch_size = 16
loader = datasets.NoDuplicatesDataLoader(
    train_data, batch_size=batch_size
)

In [ ]:
from sentence_transformers import models, SentenceTransformer
bert = models.Transformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')
pooler = models.Pooling(
    bert.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True
)

model = SentenceTransformer(modules = [bert, pooler])
model

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.7M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/383 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [ ]:
from sentence_transformers import losses
loss = losses.MultipleNegativesRankingLoss(model)

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
epochs = 3
warmup_steps = int(len(loader) * epochs * 0.1)

model.fit(
    train_objectives=[(loader, loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    output_path='retriever',
    show_progress_bar=True
)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/8144 [00:00<?, ?it/s]

Iteration:   0%|          | 0/8144 [00:00<?, ?it/s]

Iteration:   0%|          | 0/8144 [00:00<?, ?it/s]

In [ ]:
! cp -r /content/retriever /content/drive/MyDrive/

In [ ]:
val = load_dataset('squad_v2', split='validation')
val[0]

Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


{'answers': {'answer_start': [159, 159, 159, 159],
  'text': ['France', 'France', 'France', 'France']},
 'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.',
 'id': '56ddde6b9a695914005b9628',
 'question': 'In what country is Normandy located?',
 'title': 'Normans'}

In [ ]:
import pandas as pd

df = pd.DataFrame()
for row in tqdm(val):
    df = df.append({
        'question': row['question'],
        'context': row['context'],
        'id': row['id']
    }, ignore_index=True)
df.head()

  0%|          | 0/11873 [00:00<?, ?it/s]

,question,context,id
0,In what country is Normandy located?,The Normans (Norman: Nourmands; French: Norman...,56ddde6b9a695914005b9628
1,When were the Normans in Normandy?,The Normans (Norman: Nourmands; French: Norman...,56ddde6b9a695914005b9629
2,From which countries did the Norse originate?,The Normans (Norman: Nourmands; French: Norman...,56ddde6b9a695914005b962a
3,Who was the Norse leader?,The Normans (Norman: Nourmands; French: Norman...,56ddde6b9a695914005b962b
4,What century did the Normans first gain their ...,The Normans (Norman: Nourmands; French: Norman...,56ddde6b9a695914005b962c


In [ ]:
no_dupe = df.drop_duplicates(
    subset='context',
    keep='first'
)
no_dupe = no_dupe.drop(columns=['question'])
no_dupe['id'] = no_dupe['id'] + 'con'
no_dupe.head()

,context,id
0,The Normans (Norman: Nourmands; French: Norman...,56ddde6b9a695914005b9628con
9,"The Norman dynasty had a major political, cult...",56dddf4066d3e219004dad5fcon
17,"The English name ""Normans"" comes from the Fren...",56dde0379a695914005b9636con
21,"In the course of the 10th century, the initial...",56dde0ba66d3e219004dad75con
28,"Before Rollo's arrival, its populations did no...",56dde1d966d3e219004dad8dcon


In [ ]:
df = df.merge(no_dupe, how='inner', on='context')
df.head()

,question,context,id_x,id_y
0,In what country is Normandy located?,The Normans (Norman: Nourmands; French: Norman...,56ddde6b9a695914005b9628,56ddde6b9a695914005b9628con
1,When were the Normans in Normandy?,The Normans (Norman: Nourmands; French: Norman...,56ddde6b9a695914005b9629,56ddde6b9a695914005b9628con
2,From which countries did the Norse originate?,The Normans (Norman: Nourmands; French: Norman...,56ddde6b9a695914005b962a,56ddde6b9a695914005b9628con
3,Who was the Norse leader?,The Normans (Norman: Nourmands; French: Norman...,56ddde6b9a695914005b962b,56ddde6b9a695914005b9628con
4,What century did the Normans first gain their ...,The Normans (Norman: Nourmands; French: Norman...,56ddde6b9a695914005b962c,56ddde6b9a695914005b9628con


In [ ]:
ir_queries = {
    row['id_x']: row['question'] for i, row in df.iterrows()
}
ir_queries

{'56ddde6b9a695914005b9628': 'In what country is Normandy located?',
 '56ddde6b9a695914005b9629': 'When were the Normans in Normandy?',
 '56ddde6b9a695914005b962a': 'From which countries did the Norse originate?',
 '56ddde6b9a695914005b962b': 'Who was the Norse leader?',
 '56ddde6b9a695914005b962c': 'What century did the Normans first gain their separate identity?',
 '5ad39d53604f3c001a3fe8d1': "Who gave their name to Normandy in the 1000's and 1100's",
 '5ad39d53604f3c001a3fe8d2': 'What is France a region of?',
 '5ad39d53604f3c001a3fe8d3': 'Who did King Charles III swear fealty to?',
 '5ad39d53604f3c001a3fe8d4': 'When did the Frankish identity emerge?',
 '56dddf4066d3e219004dad5f': 'Who was the duke in the battle of Hastings?',
 '56dddf4066d3e219004dad60': 'Who ruled the duchy of Normandy',
 '56dddf4066d3e219004dad61': 'What religion were the Normans',
 '5ad3a266604f3c001a3fea27': 'What type of major impact did the Norman dynasty have on modern Europe?',
 '5ad3a266604f3c001a3fea28': '

In [ ]:
ir_corpus = {
    row['id_y']: row['context'] for i, row in df.iterrows()
}
ir_corpus

{'56ddde6b9a695914005b9628con': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.',
 '56dddf4066d3e219004dad5fcon': 'The Norman dynasty had a major political, cultural and military impact on medieval Europe and even the Near East. The Normans were famed for their martial spirit and eventually for their Ch

In [ ]:
ir_relevant_docs = {key: [] for key in df['id_x'].unique()}
for i, row in df.iterrows():
    ir_relevant_docs[row['id_x']].append(row['id_y'])
ir_relevant_docs = {key: set(values) for key, values in ir_relevant_docs.items()}
ir_relevant_docs

{'56ddde6b9a695914005b9628': {'56ddde6b9a695914005b9628con'},
 '56ddde6b9a695914005b9629': {'56ddde6b9a695914005b9628con'},
 '56ddde6b9a695914005b962a': {'56ddde6b9a695914005b9628con'},
 '56ddde6b9a695914005b962b': {'56ddde6b9a695914005b9628con'},
 '56ddde6b9a695914005b962c': {'56ddde6b9a695914005b9628con'},
 '5ad39d53604f3c001a3fe8d1': {'56ddde6b9a695914005b9628con'},
 '5ad39d53604f3c001a3fe8d2': {'56ddde6b9a695914005b9628con'},
 '5ad39d53604f3c001a3fe8d3': {'56ddde6b9a695914005b9628con'},
 '5ad39d53604f3c001a3fe8d4': {'56ddde6b9a695914005b9628con'},
 '56dddf4066d3e219004dad5f': {'56dddf4066d3e219004dad5fcon'},
 '56dddf4066d3e219004dad60': {'56dddf4066d3e219004dad5fcon'},
 '56dddf4066d3e219004dad61': {'56dddf4066d3e219004dad5fcon'},
 '5ad3a266604f3c001a3fea27': {'56dddf4066d3e219004dad5fcon'},
 '5ad3a266604f3c001a3fea28': {'56dddf4066d3e219004dad5fcon'},
 '5ad3a266604f3c001a3fea29': {'56dddf4066d3e219004dad5fcon'},
 '5ad3a266604f3c001a3fea2a': {'56dddf4066d3e219004dad5fcon'},
 '5ad3a2

In [ ]:
ir_eval = evaluation.InformationRetrievalEvaluator(
    ir_queries, ir_corpus, ir_relevant_docs
)

In [ ]:
ir_eval(model)

0.7385468354923839

In [ ]:
qa = SentenceTransformer('multi-qa-mpnet-base-cos-v1')

ir_eval(qa)

Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.19k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.5k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.9k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

0.7611637354436034

In [ ]:
unique_contexts = []
unique_ids = []
for row in val:
    if row['context'] not in unique_contexts:
        unique_contexts.append(row['context'])
        unique_ids.append(row['id'])
val = val.filter(lambda x: True if x['id'] in unique_ids else False)
val

  0%|          | 0/12 [00:00<?, ?ba/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 1204
})

In [ ]:
val = val.map(lambda x: {
    'encoding': model.encode(x['context']).tolist()
}, batched=True, batch_size=4)
val

  0%|          | 0/301 [00:00<?, ?ba/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'encoding'],
    num_rows: 1204
})

In [ ]:
!pip install pinecone-client

     |████████████████████████████████| 395 kB 5.2 MB/s 
     |████████████████████████████████| 266 kB 40.3 MB/s 
     |████████████████████████████████| 58 kB 3.5 MB/s 
     |████████████████████████████████| 143 kB 39.6 MB/s 


In [ ]:
import pinecone

In [ ]:
API_KEY = '2defc39d-ff30-4384-8772-9965d1b7e931'
pinecone.init(api_key=API_KEY, environment='us-west1-gcp')

In [ ]:
if 'firstqa-index' not in pinecone.list_indexes():
    pinecone.create_index(
        name='firstqa-index', dimension=model.get_sentence_embedding_dimension(), metric='cosine'
    )

In [ ]:
index = pinecone.Index('firstqa-index')

In [ ]:
from tqdm.auto import tqdm
upserts = [(v['id'], v['encoding'], {'text': v['context']}) for v in val]
for i in tqdm(range(0, len(upserts), 50)):
    i_end = i + 50
    if i_end > len(upserts): i_end = len(upserts)
    index.upsert(vectors=upserts[i:i_end])

  0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
query = "When were the Normans in Normandy?"
xq = model.encode([query]).tolist()

In [ ]:
xc = index.query(xq, top_k=2, include_metadata=True)
xc

{'results': [{'matches': [{'id': '56dddf4066d3e219004dad5f',
                           'metadata': {'text': 'The Norman dynasty had a '
                                                'major political, cultural and '
                                                'military impact on medieval '
                                                'Europe and even the Near '
                                                'East. The Normans were famed '
                                                'for their martial spirit and '
                                                'eventually for their '
                                                'Christian piety, becoming '
                                                'exponents of the Catholic '
                                                'orthodoxy into which they '
                                                'assimilated. They adopted the '
                                                'Gallo-Romance language of the '
  

In [ ]:
query = "Define chemistry?"
xq = model.encode([query]).tolist()
xc = index.query(xq, top_k=2, include_metadata=True)
xc

{'results': [{'matches': [{'id': '5737a0acc3c5551400e51f47',
                           'metadata': {'text': "Newton's laws and Newtonian "
                                                'mechanics in general were '
                                                'first developed to describe '
                                                'how forces affect idealized '
                                                'point particles rather than '
                                                'three-dimensional objects. '
                                                'However, in real life, matter '
                                                'has extended structure and '
                                                'forces that act on one part '
                                                'of an object might affect '
                                                'other parts of an object. For '
                                                'situations where lattic